In [1]:
import numpy as np
import glob
import os

In [2]:
def load_asc(asc_file):
    return np.loadtxt(asc_file, skiprows=6)

In [3]:
def combine_features(raster_folder):
    return np.array([load_asc(raster_folder+"/"+raster_folder.split('/')[-1]+"_"+str(i)+".asc") for i in range(1, 9)])

In [6]:
ls data/

metadata/  raster_data/  raster_tensors/


In [8]:
combine_features('data/raster_data/true/121147').shape

(8, 11, 11)

In [9]:
true_samples = np.array([combine_features(sample) for sample in glob.glob('data/raster_data/true/*')])
true_samples.shape

(207, 8, 11, 11)

In [10]:
false_samples = np.array([combine_features(sample) for sample in glob.glob('data/raster_data/false/*') if combine_features(sample).shape == (8, 11, 11)])
false_samples.shape

(271, 8, 11, 11)

In [11]:
true_samples.shape

(207, 8, 11, 11)

In [12]:
labels_false = np.zeros(271)
labels_true = np.ones(207)

In [13]:
train_true = true_samples

In [14]:
data = np.concatenate([true_samples, false_samples])

In [15]:
labels = np.concatenate([labels_true, labels_false])

In [16]:
data.shape

(478, 8, 11, 11)

In [17]:
labels.shape

(478,)

In [18]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.20, random_state=42)

/home/austin/anaconda3/envs/pytorch/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
X_train.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
model = Sequential()
model.add(Conv2D(filters=64, kernel_size=(2,2), padding='same', input_shape=(8, 11, 11)))
model.add(Activation('relu'))
model.add(Conv2D(64, (2,2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [ ]:
X_dev = X_train[:10]
y_dev = y_train[:10]

In [ ]:
X_dev -= X_dev.mean(axis=0) # subtract channel wise mean
X_dev /= X_dev.std(axis=0) # divide by channel wise std

In [ ]:
# X_train -= X_train.mean(axis=0) # subtract channel wise mean
# X_train /= X_train.std(axis=0) # divide by channel wise std

# X_test -= X_train.mean(axis=0) # normalize by mean calculated on training set
# X_test /= X_train.std(axis=0)

In [ ]:
model.fit(X_dev, y_dev, batch_size=32, epochs=40, shuffle=True)

In [ ]:
y_dev

In [ ]:
model.predict_proba(X_dev)

In [ ]:
model.predict_classes(X_dev)